## 问答题
1. 支持向量机的基本思想是什么？

在类之间拟合可能的最宽的街道

2. 什么是支持向量？

SVM训练后，在“街道以外”的地方增加更多的训练实例根本不会对决策边界产生影响，也就是说，它完全由位于街道边缘的实例所决定（或者“支持”）。这些实例被称为支持向量

3. 在使用 SVM 时，缩放输入值为什么很重要？

SVM的拟合类别之间可能的、最宽的街道，所以如果训练集不经缩放，SVM将趋于忽略值较小的特征

4. SVM 分类器在对实例进行分类时能输出置信度分数吗？概率呢？

SVM分类器可以输出测试实例与决策边界之间的距离，可以将其用作信心分数。如果创建SVM时，设置probability=True，可以得到概率值

5. 你如何在 LinearSVC、SVC 和 SGDClassifier 之间进行选择？

如果数据可分，可以用LinearSVC，如果数据不可分，可以用SVC，如果数据量非常大，可以用SGDClassifier，如果想输出概率值，应该使用SVC

6. 假设你已经使用 RBF 核训练了一个 SVM 分类器，但它似乎欠拟合训练集。
   你应该增大还是减小 γ（gamma）？C 呢？

增大gamma或C来降低正则化

7. ε 不敏感模型是什么意思？

如果在间隔区域内添加更多的训练实例，它不会影响模型的预测。因此，该模型被称为∈不敏感。

8. 使用核技巧有什么意义？

使用核技巧，特征数量不会出现组合爆炸式增长

## 编程题
1. 在葡萄酒数据集上训练SVM分类器，可以使用sklearn.datasets.load_wine()加载它。该数据集包含3个不同种植者生产的178个葡萄酒样本的化学分析：目标是训练一个分类模型，该模型能够根据葡萄酒的化学分析预测种植者。由于SVM分类器是二元分类器，将需要使用“一对全部”对所有三个类进行分类。能达到的精度是多少？

   "一对全部"可以复习 **8_sklearn做分类.ipynb**里的笔记，里面提到了用二元分类器做多分类问题

---

2. 提前预习 **10_支持向量机.ipynb** 最新更新的笔记 （把SVM分类用梯度下降实现）； 大概理解笔记后，尝试自己对照笔记 实现用梯度下降实现SVM分类

   并把自定义的SVM分类用于 iris data(鸢尾花数据)； 取花瓣长度 和 花瓣宽度特征， 分类 看是不是 分类2的花 （(iris.target == 2)

   对比下sklearn自带的SVM分类 和 自定义SVM分类 实现的分类效果



In [20]:
from sklearn.datasets import load_wine
import numpy as np

wine = load_wine(as_frame=True)

In [21]:
X = wine.data
y = wine.target

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

svm_clf = make_pipeline(StandardScaler(), SVC(kernel='rbf', random_state=42))
svm_clf.fit(X_train, y_train)
y_pred = svm_clf.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
accuracy

1.0

In [26]:
from sklearn.base import BaseEstimator

class MyLinearSVC(BaseEstimator):
    def __init__(self, C, eta0, n_epochs, random_state=None):
        self.C = C
        self.eta0 = eta0
        self.n_epochs = n_epochs
        self.random_state = random_state
        self._alpha = 1 / (2 * C)
        
    def eta(self):
        return self.eta0
    
    def fit(self, X, y):
        if self.random_state:
            np.random.seed(self.random_state)
        w = np.random.randn(X.shape[1], 1)
        b = 0
        t = np.array(y, dtype=np.float64).reshape(-1, 1) * 2 - 1
        m = X.shape[0]
        self.Js = []
        
        for epoch in range(self.n_epochs):
            support_vectors_idx = ((X@w + b)*t < 1).ravel()
            X_sv = X[support_vectors_idx]
            t_sv = t[support_vectors_idx]

            J = (np.sum(w * w) * self._alpha +  np.sum(1- t_sv * (X_sv@w + b))) / m
            self.Js.append(J)

            w_gradient_vector = (2*self._alpha*w  - X_sv.T @ t_sv) / m
            b_derivative = - np.sum(t_sv) / m

            w = w - self.eta() * w_gradient_vector
            b = b - self.eta() * b_derivative
            
        self.intercept_ = b
        self.coef_ = w
        support_vectors_idx = ((X@w + b)*t < 1).ravel()
        self.support_vectors_ = X[support_vectors_idx]
        return self
    
    def decision_function(self, X):
        return X.dot(self.coef_) + self.intercept_
    
    def predict(self, X):
        return self.decision_function(X) >= 0

In [27]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data[:, [2, 3]] 
y = iris.target == 2

X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.2, random_state=42)

my_svm = make_pipeline(StandardScaler(), MyLinearSVC(C=1, eta0=0.1, n_epochs=1000, random_state=42))
my_svm.fit(X_train, y_train)
y_pred = my_svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

sklearn_svm = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1, random_state=42))
sklearn_svm.fit(X_train, y_train)
y_pred = sklearn_svm.predict(X_test)
sklearn_accuracy = accuracy_score(y_test, y_pred)

accuracy, sklearn_accuracy

(0.9416666666666667, 0.9416666666666667)